In [355]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
import PublicFunctions as pf

# Read

The data was downloaded from Zillow Research (https://www.zillow.com/research/data/ --> Inventory --> New Listings (Raw, All Homes, Montly View)) as the .csv file given here. Column headers represent the last day of the month, which we simply refer to as the month, i.e., column '2018-1-31' is 'January 2018.'

In [356]:
df = pd.read_csv('Metro_new_listings_uc_sfrcondo_month.csv')
df

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,...,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31
0,102001,0,United States,country,NaN,323256.0,376081.0,476114.0,509438.0,538099.0,...,240504.0,169573.0,235076.0,247763.0,333606.0,332116.0,367751.0,376743.0,335943.0,349327.0
1,394913,1,"New York, NY",msa,NY,14880.0,22651.0,22492.0,25714.0,25627.0,...,9711.0,5829.0,9754.0,10439.0,14598.0,13820.0,14360.0,13639.0,11370.0,11234.0
2,753899,2,"Los Angeles, CA",msa,CA,8136.0,9261.0,10164.0,10597.0,11556.0,...,4806.0,3048.0,4699.0,4988.0,6370.0,6156.0,6831.0,6949.0,6658.0,6434.0
3,394463,3,"Chicago, IL",msa,IL,8645.0,11908.0,17472.0,16696.0,16869.0,...,6271.0,3899.0,6043.0,7047.0,9530.0,9649.0,10865.0,11071.0,9597.0,9942.0
4,394514,4,"Dallas, TX",msa,TX,7389.0,7894.0,10535.0,11797.0,11995.0,...,5459.0,3983.0,5188.0,5291.0,8159.0,8879.0,9467.0,10007.0,8650.0,8211.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,394968,934,"Pecos, TX",msa,TX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,6.0,9.0,5.0
921,394743,936,"Ketchikan, AK",msa,AK,9.0,11.0,17.0,11.0,17.0,...,7.0,8.0,11.0,15.0,25.0,15.0,11.0,11.0,9.0,9.0
922,753874,937,"Craig, CO",msa,CO,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,14.0,16.0,30.0,27.0,36.0,24.0,34.0
923,395188,938,"Vernon, TX",msa,TX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14.0,6.0,14.0,10.0,20.0,19.0,12.0,11.0


# Isolate the data for the team

In [357]:
team = 29
row = df.iloc[team]
row

RegionID          394355
SizeRank              29
RegionName    Austin, TX
RegionType           msa
StateName             TX
                 ...    
2023-04-30        3367.0
2023-05-31        3533.0
2023-06-30        3630.0
2023-07-31        2905.0
2023-08-31        2809.0
Name: 29, Length: 73, dtype: object

Calculate the monthly averages for years 1 thru 4. 

In [358]:
year = "2018"
y1columns = [column_name for column_name in row.index if year in column_name]
y1avg = row[y1columns].mean()
y1avg

2913.75

In [359]:
year = "2019"
y2columns = [column_name for column_name in row.index if year in column_name]
y2avg = row[y2columns].mean()
y2avg

2947.9166666666665

In [360]:
year = "2020"
y3columns = [column_name for column_name in row.index if year in column_name]
y3avg = row[y3columns].mean()
y3avg

2942.75

In [361]:
year = "2021"
y4columns = [column_name for column_name in row.index if year in column_name]
y4avg = row[y4columns].mean()
y4avg

3394.0833333333335

In [362]:
year = "2022"
y5columns = [column_name for column_name in row.index if year in column_name]
y5avg = row[y5columns].mean()
y5avg

3238.5

# Forecast Using the Decomposition Method

Calculate the seasonal index for each season each year, and then the average (over the two years) for each season.

In [363]:
values = [row[y1columns].values/y1avg,
         row[y2columns].values/y2avg,
         row[y3columns].values/y3avg,
         row[y4columns].values/y4avg,
         row[y5columns].values/y5avg]
SI = pd.DataFrame(values, columns = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], index = ['Year 1', 'Year 2', 'Year 3', 'Year 4', 'Year 5'])
SI.loc['Average'] = SI.mean()
SI

,January,February,March,April,May,June,July,August,September,October,November,December
Year 1,0.703561,0.969884,1.226255,1.366281,1.399571,1.270871,1.146289,1.072158,0.793136,0.877220,0.698069,0.476705
Year 2,0.802940,0.881979,1.191350,1.277852,1.408792,1.235110,1.148269,1.032933,0.880283,0.934898,0.702530,0.503067
Year 3,0.863138,0.907654,1.197859,1.017076,1.192082,1.201937,1.319174,1.104069,0.981735,0.955909,0.701045,0.558321
Year 4,0.675293,0.638464,1.067446,1.241572,1.167915,1.309927,1.401262,1.167031,1.096025,0.978467,0.744236,0.512362
Year 5,0.636715,0.749112,1.111317,1.269106,1.440790,1.666512,1.371931,1.033812,0.993979,0.767331,0.560753,0.398641
Average,0.736329,0.829419,1.158845,1.234377,1.321830,1.336871,1.277385,1.082001,0.949031,0.902765,0.681327,0.489819


Deseasonalize the data.

In [364]:
y_dm = np.array(row[5:65])
y_dm[0] = y_dm[0]/SI.loc['Average']['January']
y_dm[1] = y_dm[1]/SI.loc['Average']['February']
y_dm[2] = y_dm[2]/SI.loc['Average']['March']
y_dm[3] = y_dm[3]/SI.loc['Average']['April']
y_dm[4] = y_dm[4]/SI.loc['Average']['May']
y_dm[5] = y_dm[5]/SI.loc['Average']['June']
y_dm[6] = y_dm[6]/SI.loc['Average']['July']
y_dm[7] = y_dm[7]/SI.loc['Average']['August']
y_dm[8] = y_dm[8]/SI.loc['Average']['September']
y_dm[9] = y_dm[9]/SI.loc['Average']['October']
y_dm[10] = y_dm[10]/SI.loc['Average']['November']
y_dm[11] = y_dm[11]/SI.loc['Average']['December']
y_dm[12] = y_dm[12]/SI.loc['Average']['January']
y_dm[13] = y_dm[13]/SI.loc['Average']['February']
y_dm[14] = y_dm[14]/SI.loc['Average']['March']
y_dm[15] = y_dm[15]/SI.loc['Average']['April']
y_dm[16] = y_dm[16]/SI.loc['Average']['May']
y_dm[17] = y_dm[17]/SI.loc['Average']['June']
y_dm[18] = y_dm[18]/SI.loc['Average']['July']
y_dm[19] = y_dm[19]/SI.loc['Average']['August']
y_dm[20] = y_dm[20]/SI.loc['Average']['September']
y_dm[21] = y_dm[21]/SI.loc['Average']['October']
y_dm[22] = y_dm[22]/SI.loc['Average']['November']
y_dm[23] = y_dm[23]/SI.loc['Average']['December']
y_dm[24] = y_dm[24]/SI.loc['Average']['January']
y_dm[25] = y_dm[25]/SI.loc['Average']['February']
y_dm[26] = y_dm[26]/SI.loc['Average']['March']
y_dm[27] = y_dm[27]/SI.loc['Average']['April']
y_dm[28] = y_dm[28]/SI.loc['Average']['May']
y_dm[29] = y_dm[29]/SI.loc['Average']['June']
y_dm[30] = y_dm[30]/SI.loc['Average']['July']
y_dm[31] = y_dm[31]/SI.loc['Average']['August']
y_dm[32] = y_dm[32]/SI.loc['Average']['September']
y_dm[33] = y_dm[33]/SI.loc['Average']['October']
y_dm[34] = y_dm[34]/SI.loc['Average']['November']
y_dm[35] = y_dm[35]/SI.loc['Average']['December']
y_dm[36] = y_dm[36]/SI.loc['Average']['January']
y_dm[37] = y_dm[37]/SI.loc['Average']['February']
y_dm[38] = y_dm[38]/SI.loc['Average']['March']
y_dm[39] = y_dm[39]/SI.loc['Average']['April']
y_dm[40] = y_dm[40]/SI.loc['Average']['May']
y_dm[41] = y_dm[41]/SI.loc['Average']['June']
y_dm[42] = y_dm[42]/SI.loc['Average']['July']
y_dm[43] = y_dm[43]/SI.loc['Average']['August']
y_dm[44] = y_dm[44]/SI.loc['Average']['September']
y_dm[45] = y_dm[45]/SI.loc['Average']['October']
y_dm[46] = y_dm[46]/SI.loc['Average']['November']
y_dm[47] = y_dm[47]/SI.loc['Average']['December']
y_dm[48] = y_dm[48]/SI.loc['Average']['January']
y_dm[49] = y_dm[49]/SI.loc['Average']['February']
y_dm[50] = y_dm[50]/SI.loc['Average']['March']
y_dm[51] = y_dm[51]/SI.loc['Average']['April']
y_dm[52] = y_dm[52]/SI.loc['Average']['May']
y_dm[53] = y_dm[53]/SI.loc['Average']['June']
y_dm[54] = y_dm[54]/SI.loc['Average']['July']
y_dm[55] = y_dm[55]/SI.loc['Average']['August']
y_dm[56] = y_dm[56]/SI.loc['Average']['September']
y_dm[57] = y_dm[57]/SI.loc['Average']['October']
y_dm[58] = y_dm[58]/SI.loc['Average']['November']
y_dm[59] = y_dm[59]/SI.loc['Average']['December']
print("y = ",y_dm)

y =  [2784.0806865684945 3407.205461810885 3083.2415346368616 3225.107950032159
 3085.116846496077 2769.900248059269 2614.716625388807 2887.244309478417
 2435.114324232996 2831.3021278917217 2985.351344917954 2835.7387513902545
 3214.594626881769 3134.7254779576438 3030.6029302112115 3051.741182559945
 3141.856366723445 2723.52330628782 2649.9448433955426 2814.231409206716
 2734.3667985221223 3052.843765441935 3039.6571461775234
 3027.6461974886593 3449.543875065354 3220.327596778795 3041.820993449465
 2424.7043693660517 2653.8964927680818 2645.729726542164
 3039.0209400477092 3002.771050414653 3044.1563317650907
 3115.9831321437455 3027.91535131059 3354.29716957105 3112.7380163975554
 2612.6731195131592 3126.387931707067 3413.8670940556435 2998.872775750478
 3325.6755341267917 3723.2312186674153 3660.811366787455
 3919.7859308294005 3678.6988915212864 3707.471729234391
 3550.2877528204845 2800.377744245968 2924.940003663555 3105.6776611133687
 3329.6140855644744 3529.95468507864 4037.

Perform simple linear regression to extract trend.

In [365]:
model_dm = LinearRegression()
x = np.arange(1, 61).reshape((-1, 1))
model_dm.fit(x,y_dm)
r_sq_dm = model_dm.score(x, y_dm)
print('Coefficient of determination:', r_sq_dm)
intercept_dm = model_dm.intercept_
print('Intercept:', intercept_dm)
slope_dm = model_dm.coef_[0]
print('Slope:', slope_dm)

Coefficient of determination: 0.12662299339034178
Intercept: 2863.829208236648
Slope: 7.282200288287672


Extend the trend to year 6. 

In [366]:
#The forecast calculation is explicitly stated here but you can use sklearn's Predict function, as well (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).
forecast_dm = []
for t in range(61,73):
    forecast_dm.append(intercept_dm + t*slope_dm)
forecast_dm

[3308.043425822196,
 3315.3256261104834,
 3322.607826398771,
 3329.8900266870587,
 3337.172226975347,
 3344.4544272636344,
 3351.736627551922,
 3359.0188278402097,
 3366.3010281284974,
 3373.583228416785,
 3380.8654287050726,
 3388.1476289933603]

Seasonalize year 6 forecasts.

In [367]:
forecast_dm[0] = forecast_dm[0]*SI.loc['Average']['January']
forecast_dm[1] = forecast_dm[1]*SI.loc['Average']['February']
forecast_dm[2] = forecast_dm[2]*SI.loc['Average']['March']
forecast_dm[3] = forecast_dm[3]*SI.loc['Average']['April']
forecast_dm[4] = forecast_dm[4]*SI.loc['Average']['May']
forecast_dm[5] = forecast_dm[5]*SI.loc['Average']['June']
forecast_dm[6] = forecast_dm[6]*SI.loc['Average']['July']
forecast_dm[7] = forecast_dm[7]*SI.loc['Average']['August']
forecast_dm[8] = forecast_dm[8]*SI.loc['Average']['September']
forecast_dm[9] = forecast_dm[9]*SI.loc['Average']['October']
forecast_dm[10] = forecast_dm[10]*SI.loc['Average']['November']
forecast_dm[11] = forecast_dm[11]*SI.loc['Average']['December']
forecast_dm

[2435.80908256434,
 2749.7931440884304,
 3850.388505200593,
 4110.340615454126,
 4411.174363480553,
 4471.104962294022,
 4281.45835281816,
 3634.460299644157,
 3194.7254379752067,
 3045.552308560646,
 2303.474361130887,
 1659.5806134696074]

# Forecast Using Multiple Linear Regression

Populate independent variables

In [368]:
x = np.arange(1, 61).reshape((-1, 1))#Must change this to add the dummy binary variables.
x

array([[ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25],
       [26],
       [27],
       [28],
       [29],
       [30],
       [31],
       [32],
       [33],
       [34],
       [35],
       [36],
       [37],
       [38],
       [39],
       [40],
       [41],
       [42],
       [43],
       [44],
       [45],
       [46],
       [47],
       [48],
       [49],
       [50],
       [51],
       [52],
       [53],
       [54],
       [55],
       [56],
       [57],
       [58],
       [59],
       [60]])

Run multiple linear regression.

In [369]:
model_mlr = LinearRegression()
model_mlr.fit(x,y_dm) #Must change this. Basically, you need to extend the simple linear regression model to take multiple independent variables. You are referred to https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html for the documentation of the package we use for this purpose. Make sure you use the correct y values, too.

LinearRegression()

Calculate the forecasts.

In [370]:
forecast_mlr = []
#Must add this. Decide first whether you want to calculate the forecasts explicitly or via sklearn's Predict function (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) and then execute.
forecast_mlr

[]

# Calculate Forecast Accuracy

In [371]:
actual_Year6 = []
for t in range(65,73):
    actual_Year6.append(row[t])
actual_Year6

[2102.0, 2135.0, 3511.0, 3367.0, 3533.0, 3630.0, 2905.0, 2809.0]

In [372]:
deviation_dm = []
deviation_mlr = []
for t in range(8):
    deviation_dm.append(np.random.normal(0,1,1)) #Must change this, it is curently just random
    deviation_mlr.append(np.random.normal(0,1,1)) #Must change this, it is curently just random
MAD_dm = pf.CalculateMeanAbsoluteDeviation(deviation_dm)
MAD_mlr = pf.CalculateMeanAbsoluteDeviation(deviation_mlr)
print("MAD_DM:",MAD_dm)
print("MAD_MLR:",MAD_mlr)

MAD_DM: 0.8154449389270688
MAD_MLR: 0.5635365895740245
